In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os

In [2]:
cd ../

D:\git_repos\lab_data_science


In [3]:
#TRAIN - VAL

In [4]:
import src.config as cfg
from src.utils import save_as_pickle, load_as_pickle

In [5]:
# raw_path = 'data/raw/'
# train = pd.read_csv(os.path.join(raw_path, 'train.csv'))
# test = pd.read_csv(os.path.join(raw_path, 'test.csv'))

In [6]:
# TARGET_COLS = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']
# ID_COL = 'ID'
# EDU_COL = 'Образование'
# SEX_COL = 'Пол'
# CAT_COLS = [
#     'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 
#     'Профессия', 'Статус Курения', 'Алкоголь',
#     'Время засыпания', 'Время пробуждения'
# ]
# OHE_COLS = [
#     'Пол', 'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
#     'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма', 'Туберкулез легких ', 'ВИЧ/СПИД',
#     'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы','Пассивное курение', 'Сон после обеда', 
#     'Спорт, клубы', 'Религия, клубы'
# ]
# REAL_COLS = ['Возраст курения', 'Сигарет в день', 'Возраст алког', 'Частота пасс кур']

In [7]:
# def set_idx(df: pd.DataFrame, idx_col: str) -> pd.DataFrame:
#     df = df.set_index(idx_col)
#     return df

In [8]:
# train = train.set_index(ID_COL)
# test = test.set_index(ID_COL)

In [9]:
# train, target = train.drop(TARGET_COLS, axis=1), train[TARGET_COLS]

In [10]:
# train.pipe()

In [11]:
# def preprocess(df: pd.DataFrame) -> pd.DataFrame:
#     df = set_idx(df, ID_COL)
#     df = drop_unnecesary_id(df)
#     df = fill_sex(df)
#     df = cast_types(df)
#     return df
    
    

In [12]:
# def drop_unnecesary_id(df: pd.DataFrame) -> pd.DataFrame:
#     if 'ID_y' in df.columns:
#         df = df.drop('ID_y', axis=1)
#     return df

In [13]:
# train = drop_unnecesary_id(train)

In [14]:
# def add_ord_edu(df: pd.DataFrame) -> pd.DataFrame:
#     df[f'{EDU_COL}_ord'] = df[EDU_COL].str.slice(0, 1).astype(np.int8).values
#     return df

In [15]:
# train = add_ord_edu(train)
# test = add_ord_edu(test)

In [16]:
# def fill_sex(df: pd.DataFrame) -> pd.DataFrame:
#     most_freq = df[SEX_COL].value_counts().index[0]
#     df[SEX_COL] = df[SEX_COL].fillna(most_freq)
#     return df

In [17]:
# def cast_types(df: pd.DataFrame) -> pd.DataFrame:
#     df[CAT_COLS] = df[CAT_COLS].astype('category')

#     ohe_int_cols = train[OHE_COLS].select_dtypes('number').columns
#     df[ohe_int_cols] = df[ohe_int_cols].astype(np.int8)

#     df[REAL_COLS] = df[REAL_COLS].astype(np.float32)
#     return df
    

In [18]:
from sklearn.metrics import r2_score, mean_squared_error, precision_score, recall_score, roc_auc_score, f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [19]:
train_data_path = 'data/interim/train.pkl'
target_data_path = 'data/interim/target.pkl'

In [20]:
def load_splitted_data(train_path, target_path):
    train = load_as_pickle(train_path)
    target = load_as_pickle(target_path)
    return train, target

In [21]:
train_data, target_data = load_splitted_data(train_data_path, target_data_path)

In [22]:
x_train, x_test, y_train, y_test = train_test_split(train_data, target_data, test_size=0.25)

In [23]:
path_to_splitted_train_data = 'data/processed/train.pkl'
path_to_splitted_train_data_target = 'data/processed/train_target.pkl'
path_to_splitted_val_data = 'data/processed/val.pkl'
path_to_splitted_val_data_target = 'data/processed/val_target.pkl'

In [24]:
save_as_pickle(x_train, path_to_splitted_train_data)
save_as_pickle(y_train, path_to_splitted_train_data_target)
save_as_pickle(x_test, path_to_splitted_val_data)
save_as_pickle(y_test, path_to_splitted_val_data_target)

In [25]:
#FEATURE GENERATION

In [26]:
import category_encoders as ce

In [27]:
def second_hand_smoke_count(value):
    if value == '1-2 раза в неделю':
        return 1.5
    if value == '2-3 раза в день':
        return 17.5
    if value == '3-6 раз в неделю':
        return 4.5
    if value == '4 и более раз в день':
        return 28.0
    if value == 'не менее 1 раза в день':
        return 7.0
    return 0.0

In [28]:
def fill_real_cols_with_zero(df):
    df = df.fillna(0.)
    return df

In [29]:
def fill_real_cols(data):
    data.loc[:, cfg.REAL_COLS[:-1]] = fill_real_cols_with_zero(data.loc[:, cfg.REAL_COLS[:-1]])
    data['Частота пасс кур'] = [second_hand_smoke_count(i) for i in data['Частота пасс кур']]
    return data

In [30]:
x_train = fill_real_cols(x_train)

In [31]:
# def target_encode_multiclass(X,y): #X,y are pandas df and series
#     y=y.astype(str)  #convert to string to onehot encode
#     enc=ce.OneHotEncoder().fit(y)
#     y_onehot=enc.transform(y) 

#     class_names=y_onehot.columns  #names of onehot encoded columns

#     X_obj=X.select_dtypes('object') #separate categorical columns
#     X=X.select_dtypes(exclude='object')

#     for class_ in class_names:

#         enc=ce.TargetEncoder()
#         enc.fit(X_obj,y_onehot[class_]) #convert all categorical
#         temp=enc.transform(X_obj)       #columns for class_
#         temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
#         X=pd.concat([X,temp],axis=1)    #add to original dataset

#     return X

In [44]:
# class TargetEncoder(BaseEstimator, TransformerMixin):
class TargetEncoder():
    
    def __init__(self, cat_cols, num_classes=1):
        self.te = [ce.TargetEncoder(drop_invariant=False)] * num_classes
        self.cat_cols = cat_cols
        
        
    def fit(self, x, y):
        for target_num in range(num_classes):
            target_col = cfg.TARGET_COLS[target_num]
            self.te[target_num].fit(x[self.cat_cols], y_train[target_col])

        
    def transform(self, x, y=None):
        target_encoded_x = x.copy()
        for target_num in range(num_classes):
            target_col = cfg.TARGET_COLS[target_num]
            encoded_cols = self.te[target_num].transform(x[self.cat_cols])
            
            target_encoded_x[[col + "/" + target_col for col in self.cat_cols]] = encoded_cols

        target_encoded_x = target_encoded_x.drop(self.cat_cols, axis=1)
        return target_encoded_x
            
    def fit_transform(self, x, y=None):
        self.fit(x, y)
        return self.transform(x, y)

In [45]:
target_encoder = TargetEncoder(cfg.CAT_COLS, 5)

C:\git\lab_data_science\venv\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\git\lab_data_science\venv\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [34]:
# def target_encoding(cat_features, x_train, y_train=None, test=False):
#     target_encoded_x_train = x_train.copy()
#     for target_num in range(num_classes):
#         target_col = cfg.TARGET_COLS[target_num]
#         if test:
#             encoded_cols = te[target_num].transform(x_train[cat_features])
#         else:
#             encoded_cols = te[target_num].fit_transform(x_train[cat_features], y_train[target_col])
            
#         target_encoded_x_train[[col + "/" + target_col for col in cfg.CAT_COLS]] = encoded_cols

#     target_encoded_x_train = target_encoded_x_train.drop(cat_features, axis=1)
#     return target_encoded_x_train

In [42]:
num_classes = 5

In [36]:

#     target_encoded_x_train.insert(loc=target_encoded_x_train.columns[:-1],
#                                   column=[col + "/" + target_col for col in cfg.CAT_COLS],
#                                   value=encoded_cols,
#                                   allow_duplicates = False)
        

In [49]:
target_encoded_x_train = target_encoder.fit_transform(x_train, y_train)

target_encoded_x_test = target_encoder.transform(x_test, y_test)
target_encoded_x_test

,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,...,Семья/Прочие заболевания сердца,Этнос/Прочие заболевания сердца,Национальность/Прочие заболевания сердца,Религия/Прочие заболевания сердца,Образование/Прочие заболевания сердца,Профессия/Прочие заболевания сердца,Статус Курения/Прочие заболевания сердца,Алкоголь/Прочие заболевания сердца,Время засыпания/Прочие заболевания сердца,Время пробуждения/Прочие заболевания сердца
ID,,,,,,,,,,,,,,,,,,,,,
54-002-154-02,1,0,0,0,0,0,0,0,0,0,...,9.433962e-02,0.095578,0.095168,9.900990e-02,0.071429,0.000000,0.087248,0.084926,0.096774,0.108108
54-102-005-01,1,0,0,0,0,0,0,0,0,0,...,1.166667e-01,0.095578,0.000004,1.053052e-08,0.088710,0.118644,0.084967,0.084926,0.125000,0.093575
54-501-028-02,1,0,0,0,0,0,0,0,0,0,...,3.220577e-15,0.095578,0.095168,9.900990e-02,0.103858,0.118644,0.087248,0.084926,0.078261,0.108108
54-101-006-02,0,0,0,0,0,0,0,0,0,0,...,8.421053e-02,0.095578,0.000004,1.053052e-08,0.103858,0.112676,0.087248,0.112069,0.004438,0.077670
54-102-361-01,1,0,0,0,0,0,0,0,0,0,...,3.220577e-15,0.095578,0.095168,9.900990e-02,0.088710,0.118644,0.099034,0.084926,0.096774,0.089286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54-501-040-02,1,0,0,0,0,0,0,0,0,0,...,1.166667e-01,0.095578,0.095168,9.900990e-02,0.103858,0.072917,0.084967,0.084926,0.096774,0.067073
54-102-106-01,1,0,0,0,0,0,0,0,0,0,...,1.165049e-01,0.000004,0.000004,1.053052e-08,0.103858,0.112676,0.099034,0.112069,0.078261,0.107143
54-102-319-01,1,0,0,0,0,0,0,0,0,0,...,8.421053e-02,0.095578,0.095168,9.900990e-02,0.071429,0.000000,0.087248,0.108527,0.078261,0.067073


In [ ]:
train_data_for_catboost, 

In [ ]:
#MODELS

In [ ]:
import catboost

In [ ]:
def get_indexes_of_cat_columns(df: pd.DataFrame, col_names: str):
    idx = []
    for col in col_names:
        idx.append(df.columns.tolist().index(col))
        
    return idx

In [ ]:
cfg.CAT_COLS

In [ ]:
get_indexes_of_cat_columns(train_data, cfg.CAT_COLS)

In [ ]:
cat_idx = get_indexes_of_cat_columns(train_data, cfg.CAT_COLS)
cat_idx

In [ ]:
# for i in range(100, 500, 100):
i = 100
boosting_model = catboost.CatBoostClassifier(
    loss_function='MultiLogloss',
    cat_features=cat_idx
)
    

In [ ]:
#Так как крайне важно правильно предсказать наличие смертельной болезни, для нашей задачи хорошо подойдет метрика recall, показывающая долю верно предсказанных болезней. Также, чтобы выявить количество ложно положительных результатов(то есть случаев, когда модель выявила болезнь у здорового человека), можно использовать метрику presicion. В ккачестве метрики, объединяющей две предыдущие, возьмем roc_auc. 

In [ ]:
# train_pool = catboost.Pool(x_train, y_train)
# val_pool = catboost.Pool(x_test, y_test)

# boosting_model.fit(train_data, target_data)
boosting_model = boosting_model.fit(x_train, y_train, metric_period=10, verbose=50)

In [ ]:
preds = boosting_model.predict(x_test)

In [ ]:
def count_metrics(y_test, preds):
    print(precision_score(y_test, preds, average='micro'))
    print(recall_score(y_test, preds, average='micro'))
    print(f1_score(y_test, preds, average='micro'))
    print(roc_auc_score(y_test, preds, average='micro'))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
fCl_target_encode = MultiOutputClassifier(LogisticRegression()).fit(target_encoded_x_train, y_train)
# fCl = MultiOutputClassifier(RandomForestClassifier()).fit(x_train, y_train)

In [ ]:
target_encoded_test = target_encoding(cfg.CAT_COLS, x_test, test=True)
target_encoded_test = fill_real_cols(target_encoded_test)
preds = fCl_target_encode.predict(target_encoded_test)

In [ ]:
count_metrics(y_test, preds)